<a href="https://colab.research.google.com/github/rajatgupta3121/Rajat_gupta_143/blob/main/FinalTechExpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install streamlit

In [ ]:
pip install SpeechRecognition

In [6]:
pip install gtts

In [7]:
import streamlit as st
import json
import nltk
import string
import random
import speech_recognition as sr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gtts import gTTS
import os

In [19]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
# Load intents from JSON file
with open("/content/intents.json", "r") as file:
    intents_data = json.load(file)

In [10]:
# Preprocess intents data
intents = intents_data["intents"]

In [11]:
# Tokenization and lemmatization
lemmer = nltk.stem.WordNetLemmatizer()


In [12]:
def LemTokens(tokens):
    return [lemmer.lemmatize(token.lower()) for token in tokens]

In [13]:
remove_punc_dict = dict((ord(punct), None) for punct in string.punctuation)

In [14]:
def LemNormalize(text):
    return ' '.join(LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict))))


In [15]:
# Initialize dictionaries to hold patterns and responses
patterns_responses = {}


In [20]:
# Extract patterns and responses from intents
for intent in intents:
    for pattern in intent["patterns"]:
        # Join the pattern list into a single string
        pattern_text = ' '.join(pattern)
        patterns_responses[LemNormalize(pattern_text)] = intent["responses"]

In [21]:
# Create TF-IDF vectorizer
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words="english",token_pattern=None)

# Convert text data to TF-IDF vectors
tfidf_matrix = TfidfVec.fit_transform(patterns_responses.keys())


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens [' ', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn(


In [22]:
# Function to get response from the bot
def get_response(user_response):
    # Convert user input to TF-IDF vector
    user_tfidf = TfidfVec.transform([user_response])

    # Calculate cosine similarity between user input and patterns
    similarity_scores = cosine_similarity(user_tfidf, tfidf_matrix)

    # Get index of the most similar pattern
    max_sim_index = similarity_scores.argmax()

    # Get corresponding response for the most similar pattern
    response = list(patterns_responses.values())[max_sim_index]

    return random.choice(response)  # Randomly select a response

In [23]:
# Function for speech to text conversion
def speech_to_text():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        st.write("Listening...")
        audio = r.listen(source)

    try:
        user_input = r.recognize_google(audio)
        st.text_input("You:", user_input)
        return user_input
    except sr.UnknownValueError:
        st.write("Sorry, I couldn't understand what you said.")
    except sr.RequestError:
        st.write("Sorry, my speech recognition service is unavailable at the moment.")


In [24]:
# Function for text to speech conversion
def text_to_speech(text):
    tts = gTTS(text=text, lang='en')
    tts.save("output.mp3")
    os.system("start output.mp3")


In [28]:
# Main function to interact with the bot
def main():
    st.title("Chat Bot")
    st.write("Hello! I am the Chat Bot. You can start chatting with me.")

    # Display image of cute chatbot
    st.image("/content/bot.png", caption="Cute Chatbot", use_column_width=True)

    speech_input = st.button("Speak")
    if speech_input:
        user_input = speech_to_text()
        if user_input:
            bot_response = get_response(user_input.lower())
            st.text_area("Bot:", value=bot_response, height=200)
            text_to_speech(bot_response)

    user_input = st.text_input("You:", "")
    if user_input:
        bot_response = get_response(user_input.lower())
        st.text_area("Bot:", value=bot_response, height=200)
        text_to_speech(bot_response)


In [29]:
if __name__ == "__main__":
    main()
